In [273]:
import requests
import pandas as pd

# Define your FRED API key
API_KEY = "dc6a0313beabf8a2845ed521ddb56d7b"

# Define the FRED API endpoint for FOMC meeting dates
FRED_ENDPOINT = 'https://api.stlouisfed.org/fred/series/observations'

# Parameters for the API request
params = {
    'series_id': 'FEDFUNDS',  # This series represents the federal funds rate, which is influenced by FOMC meetings
    'api_key': API_KEY,
    'file_type': 'json'
}

# Make the API request
response = requests.get(FRED_ENDPOINT, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Extract observation dates
    observations = data['observations']
    
    
    # Create a DataFrame
    #df = pd.DataFrame(dates, columns=['Date'])
    df = pd.DataFrame(observations)
    df = df[['date', 'value']]
    df.columns = ['date', 'ffr']
    df['date'] = pd.to_datetime(df['date'])
    # Print the DataFrame
    print(df)
else:
    print(f"Error fetching data: {response.status_code}")

# Optionally, save the data to a CSV file
#df.to_csv('fomc_meeting_dates.csv', index=False)


          date   ffr
0   1954-07-01  0.80
1   1954-08-01  1.22
2   1954-09-01  1.07
3   1954-10-01  0.85
4   1954-11-01  0.83
..         ...   ...
835 2024-02-01  5.33
836 2024-03-01  5.33
837 2024-04-01  5.33
838 2024-05-01  5.33
839 2024-06-01  5.33

[840 rows x 2 columns]


In [235]:
def complete_month(month_abbr):
  """Completes a month name from its abbreviation.

  Args:
    month_abbr: The first three letters of a month name.

  Returns:
    The complete month name if found, otherwise None.
  """

  for month_number, month_name in enumerate(calendar.month_name):
    if month_name.lower().startswith(month_abbr.lower()):
      return month_name
  return None

def month_str_to_num(month_str):
  """Converts a month string to its corresponding number.

  Args:
    month_str: The month name as a string (e.g., "January", "Feb").

  Returns:
    The month number (1-12) if found, otherwise None.
  """

  month_str = month_str.lower()
  for month_num, month_name in enumerate(calendar.month_name):
    if month_name.lower().startswith(month_str):
      return month_num
  return None

In [283]:
from bs4 import BeautifulSoup
import calendar
import datetime
import re 
import requests

year = 2024 # in your code take the today date and extract the year to automate the entire process 

# URL of the FOMC meeting calendar page
url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the header with the text "2024 FOMC Meetings"
    header = soup.find('h4', string=f"{year} FOMC Meetings")

    parent_div1 = header.find_parent('div')

    main_parent= parent_div1.find_parent('div')

    months = main_parent.find_all('div', re.compile("fomc-meeting__month "))
    days = main_parent.find_all('div', "fomc-meeting__date col-xs-4 col-sm-9 col-md-10 col-lg-1")
    
    dates = {}
    for i in range(len(months)):
        
        month = months[i].text
        
        if '/' in month:
            m1, m2 = month.split("/")
            d1, d2 = days[i].text.split('-')
            
            #m1 = complete_month(m1)
            m2 = complete_month(m2)
            
            #dates[m1] = d1 # just remove this line if not interested in day 1 of the FED 
            dates[m2] = d2
            
            continue
        
        else:
            dates[months[i].text] = days[i].text.split('-')

    #dates = pd.DataFrame(dates, index = [1,2] ,columns = dates.keys())
    
dates  
    

{'January': ['30', '31'],
 'March': ['19', '20*'],
 'May': '1',
 'June': ['11', '12*'],
 'July': ['30', '31'],
 'September': ['17', '18*'],
 'November': ['6', '7'],
 'December': ['17', '18*']}

In [270]:
today = datetime.date.today()
month = calendar.month_name[today.month]
day = today.day
print("today: ", today)
print("Month and Day: ",  (year, month, day ))

today:  2024-08-01
Month and Day:  (2024, 'August', 1)


In [271]:

try:
    days = dates[month]
    print(f'Fed Decision Day on {month}', days, year)
    try:
        #fed_decision_date = datetime.date(year, month_str_to_num(month),int(days[1].replace('*','')))
        fed_decision_date = datetime.date(year, today.month, int(days[1].replace('*','')))
    except IndexError:
        #fed_decision_date = datetime.date(year, month_str_to_num(month),int(days[0].replace('*','')))
        fed_decision_date = datetime.date(year, today.month, int(days[0].replace('*','')))
    
except KeyError:
    print(f'No FED in {month}.')
    next_month = calendar.month_name[(today + datetime.timedelta(days=32)).month]
    days = dates[next_month]
    try:
        fed_decision_date = datetime.date(year, today.month + 1, int(days[1].replace('*','')))
        print(f'Fed Decision on {next_month}', days[1], year)
    except IndexError:
        fed_decision_date = datetime.date(year, today.month + 1, int(days[0].replace('*','')))
        print(f'Fed Decision on {next_month}', days[0], year)

No FED in August.
Fed Decision on September 18* 2024


In [272]:
import datetime

def count_weekdays(start_date, end_date):
  """Counts the number of weekdays between two dates.

  Args:
    start_date: The start date as a datetime.date object.
    end_date: The end date as a datetime.date object.

  Returns:
    The number of weekdays between the two dates.
  """

  delta = end_date - start_date
  total_days = delta.days + 1  # Include both start and end dates

  # Calculate the number of full weeks
  full_weeks, remaining_days = divmod(total_days, 7)
  weekdays = full_weeks * 5  # 5 weekdays per week

  # Handle the remaining days
  start_weekday = start_date.weekday()
  end_weekday = end_date.weekday()

  if start_weekday < 5:  # Start day is not a weekend
    weekdays += min(end_weekday + 1, 5) - start_weekday
  else:
    weekdays += max(0, end_weekday - 4)

  return weekdays

num_weekdays = count_weekdays(today, fed_decision_date)
print("Trading days until FED decision: ", num_weekdays) 
print('Calendar Days until FED decision: ', (fed_decision_date - today).days)


Trading days until FED decision:  35
Calendar Days until FED decision:  48


#### Historical FOMC Meeting Dates

In [ ]:
dates = {}

    for header in headers:

        items = header.text.split(' ')
        
        month = items[0]
        days = items[1]
        
        try:
            d1, d2 = days.split("-")
        except ValueError:
            d2 = days 
        
        if '/' in month:
            
            m1,m2 = month.split("/")
            dates[complete_month(m2)] = d2 
            
        else:
            
            dates[month] = d2 


In [472]:
#df = pd.DataFrame(index = calendar.month_name[1:])

df = pd.DataFrame()
for year in range(1990, 2019, 1):
    print(year)
    # URL of the FOMC meeting calendar page
    url = f"https://www.federalreserve.gov/monetarypolicy/fomchistorical{year}.htm"

    # Make a GET request to fetch the raw HTML content
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the header with the text "2024 FOMC Meetings"
        #headers = soup.find_all('h5', "panel-heading panel-heading--shaded")
        headers = soup.find_all('h5')
        
        dates= {}
        dates_list = []
        
        for header in headers:

            items = header.text.split(' ')
            items = header.text
            
            month = items[0]
            days = items[1]
            #print(month, days)
            
            dates_list.append(items)
            
        dates_list = pd.DataFrame(dates_list)
        df = pd.concat([df, dates_list], axis = 0)
        #     try:
        #         d1, d2 = days.split("-")
        #     except ValueError:
        #         d2 = days 
                
        #     if '/' in month:
                
        #         m1,m2 = month.split("/")
        #         dates[complete_month(m2)] = d2 
                
        #         day1 = datetime.date(year, month_str_to_num(complete_month(m1)), int(d1))
        #         day2 = datetime.date(year, month_str_to_num(complete_month(m2)), int(d2))
        #         dates_list.append(day1)
        #         dates_list.append(day2)
                
        #     else:
                
        #         dates[month] = d2 
        #         day1 = datetime.date(year, month_str_to_num(month), int(d1))
        #         day2 = datetime.date(year, month_str_to_num(month), int(d2))
        #         dates_list.append(day1)
        #         dates_list.append(day2)
            

        # df1= pd.DataFrame(dates, index = [1],columns = dates.keys()).transpose()
        # df1.columns = [str(year)]
        
        # df = df.merge(df1, how='left', left_index= True, right_index=True)
        
        # if year == 2010:
        #     # print(df1)
        #     print(dates)

    else: 
        print("Unsuccessful Call: ", response.status_code)


df.columns = ['dates']

1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [473]:
df

,dates
0,February 6-7 Meeting - 1990
1,March 27 Meeting - 1990
2,April 11 Conference Call - 1990
3,May 15 Meeting - 1990
4,July 2-3 Meeting - 1990
...,...
3,June 12-13 Meeting - 2018
4,Jul/Aug 31-1 Meeting - 2018
5,September 25-26 Meeting - 2018
6,November 7-8 Meeting - 2018


In [474]:

# Define a function to extract month, day(s), and year
def extract_date_components(date_str):
    # Extract the month
    month_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December)'
    month_match = re.search(month_pattern, date_str)
    month = month_match.group(0) if month_match else 'None'

    # Extract the day(s)
    day_pattern = r'(\d{1,2}(?:-\d{1,2})?)'
    day_match = re.search(day_pattern, date_str)
    day = day_match.group(0) if day_match else 'None'

    # Extract the year
    year_pattern = r'\b(\d{4})\b'
    year_match = re.search(year_pattern, date_str)
    year = year_match.group(0) if year_match else 'None'

    return month, day, year

# Apply the function to the 'dates' column and create new columns
df[['month', 'day', 'year']] = df['dates'].apply(lambda x: pd.Series(extract_date_components(x)))

# Display the DataFrame
df

,dates,month,day,year
0,February 6-7 Meeting - 1990,February,6-7,1990
1,March 27 Meeting - 1990,March,27,1990
2,April 11 Conference Call - 1990,April,11,1990
3,May 15 Meeting - 1990,May,15,1990
4,July 2-3 Meeting - 1990,July,2-3,1990
...,...,...,...,...
3,June 12-13 Meeting - 2018,June,12-13,2018
4,Jul/Aug 31-1 Meeting - 2018,None,31-1,2018
5,September 25-26 Meeting - 2018,September,25-26,2018
6,November 7-8 Meeting - 2018,November,7-8,2018


In [484]:
df = df.reset_index(drop = True)
df

,dates,month,day,year,month_day1
0,February 6-7 Meeting - 1990,February,6-7,1990,nan
1,March 27 Meeting - 1990,March,27,1990,nan
2,April 11 Conference Call - 1990,April,11,1990,nan
3,May 15 Meeting - 1990,May,15,1990,nan
4,July 2-3 Meeting - 1990,July,2-3,1990,nan
...,...,...,...,...,...
292,June 12-13 Meeting - 2018,June,12-13,2018,nan
293,Jul/Aug 31-1 Meeting - 2018,None,31-1,2018,"[Jul, Aug 31-1 Meeting - 2018]"
294,September 25-26 Meeting - 2018,September,25-26,2018,nan
295,November 7-8 Meeting - 2018,November,7-8,2018,nan


In [501]:
import numpy as np 

df['month_day1'] = np.where(df['dates'].str.contains('/'), df['dates'].str.split(' '), 'nan')
df['month_day1'] = df['month_day1'].apply(lambda x: x[0])
df['month_day1'] = np.where(df['dates'].str.contains('/'), df['month_day1'].str.split('/'), 'nan')
df['month_day1'] = df['month_day1'].apply(lambda x: x[0])


df['month_day2'] = np.where(df['dates'].str.contains('/'), df['dates'].str.split(' '), 'nan')
df['month_day2'] = df['month_day2'].apply(lambda x: x[0])
df['month_day2'] = np.where(df['dates'].str.contains('/'), df['month_day2'].str.split('/'), 'nan')
df['month_day2'] = df['month_day2'].apply(lambda x: x[1])
df

,dates,month,day,year,month_day1,month_day2
0,February 6-7 Meeting - 1990,February,6-7,1990,n,a
1,March 27 Meeting - 1990,March,27,1990,n,a
2,April 11 Conference Call - 1990,April,11,1990,n,a
3,May 15 Meeting - 1990,May,15,1990,n,a
4,July 2-3 Meeting - 1990,July,2-3,1990,n,a
...,...,...,...,...,...,...
292,June 12-13 Meeting - 2018,June,12-13,2018,n,a
293,Jul/Aug 31-1 Meeting - 2018,None,31-1,2018,Jul,Aug
294,September 25-26 Meeting - 2018,September,25-26,2018,n,a
295,November 7-8 Meeting - 2018,November,7-8,2018,n,a


In [509]:
df['day1'] = df['day'].str.split('-').apply(lambda x: x[0] if len(x) > 1 else 'nan')
df['day2'] = df['day'].str.split('-').apply(lambda x: x[1] if len(x)> 1 else x[0])

df

,dates,month,day,year,month_day1,month_day2,day1,day2
0,February 6-7 Meeting - 1990,February,6-7,1990,n,a,6,7
1,March 27 Meeting - 1990,March,27,1990,n,a,nan,27
2,April 11 Conference Call - 1990,April,11,1990,n,a,nan,11
3,May 15 Meeting - 1990,May,15,1990,n,a,nan,15
4,July 2-3 Meeting - 1990,July,2-3,1990,n,a,2,3
...,...,...,...,...,...,...,...,...
292,June 12-13 Meeting - 2018,June,12-13,2018,n,a,12,13
293,Jul/Aug 31-1 Meeting - 2018,None,31-1,2018,Jul,Aug,31,1
294,September 25-26 Meeting - 2018,September,25-26,2018,n,a,25,26
295,November 7-8 Meeting - 2018,November,7-8,2018,n,a,7,8


In [512]:
df['month_day1'] = df['month_day1'].apply(lambda x: complete_month(x) if len(x) > 1 else x)
df['month_day2'] = df['month_day2'].apply(lambda x: complete_month(x) if len(x) > 1 else x)
df

,dates,month,day,year,month_day1,month_day2,day1,day2
0,February 6-7 Meeting - 1990,February,6-7,1990,n,a,6,7
1,March 27 Meeting - 1990,March,27,1990,n,a,nan,27
2,April 11 Conference Call - 1990,April,11,1990,n,a,nan,11
3,May 15 Meeting - 1990,May,15,1990,n,a,nan,15
4,July 2-3 Meeting - 1990,July,2-3,1990,n,a,2,3
...,...,...,...,...,...,...,...,...
292,June 12-13 Meeting - 2018,June,12-13,2018,n,a,12,13
293,Jul/Aug 31-1 Meeting - 2018,None,31-1,2018,July,August,31,1
294,September 25-26 Meeting - 2018,September,25-26,2018,n,a,25,26
295,November 7-8 Meeting - 2018,November,7-8,2018,n,a,7,8


In [528]:
timestamps = []


for i, row in df.iterrows():
    
    if '/' in row['dates']:
        timestamp = datetime.date(int(row['year']), month_str_to_num(row['month_day2']), int(row['day2']))
        timestamps.append(timestamp)
        
    else: 
        timestamp = datetime.date(int(row['year']), month_str_to_num(row['month']), int(row['day2']))
        timestamps.append(timestamp)

        


timestamps

[datetime.date(1990, 2, 7),
 datetime.date(1990, 3, 27),
 datetime.date(1990, 4, 11),
 datetime.date(1990, 5, 15),
 datetime.date(1990, 7, 3),
 datetime.date(1990, 8, 21),
 datetime.date(1990, 9, 7),
 datetime.date(1990, 9, 17),
 datetime.date(1990, 10, 2),
 datetime.date(1990, 11, 13),
 datetime.date(1990, 12, 7),
 datetime.date(1990, 12, 18),
 datetime.date(1991, 1, 9),
 datetime.date(1991, 2, 1),
 datetime.date(1991, 2, 6),
 datetime.date(1991, 3, 26),
 datetime.date(1991, 4, 12),
 datetime.date(1991, 4, 30),
 datetime.date(1991, 5, 1),
 datetime.date(1991, 5, 14),
 datetime.date(1991, 6, 10),
 datetime.date(1991, 6, 24),
 datetime.date(1991, 7, 3),
 datetime.date(1991, 8, 5),
 datetime.date(1991, 8, 20),
 datetime.date(1991, 9, 13),
 datetime.date(1991, 10, 1),
 datetime.date(1991, 10, 30),
 datetime.date(1991, 11, 5),
 datetime.date(1991, 12, 2),
 datetime.date(1991, 12, 17),
 datetime.date(1992, 1, 9),
 datetime.date(1992, 2, 5),
 datetime.date(1992, 3, 11),
 datetime.date(1992, 

In [529]:
df['timestamp'] = timestamps

df

,dates,month,day,year,month_day1,month_day2,day1,day2,timestamp
0,February 6-7 Meeting - 1990,February,6-7,1990,n,a,6,7,1990-02-07
1,March 27 Meeting - 1990,March,27,1990,n,a,nan,27,1990-03-27
2,April 11 Conference Call - 1990,April,11,1990,n,a,nan,11,1990-04-11
3,May 15 Meeting - 1990,May,15,1990,n,a,nan,15,1990-05-15
4,July 2-3 Meeting - 1990,July,2-3,1990,n,a,2,3,1990-07-03
...,...,...,...,...,...,...,...,...,...
292,June 12-13 Meeting - 2018,June,12-13,2018,n,a,12,13,2018-06-13
293,Jul/Aug 31-1 Meeting - 2018,None,31-1,2018,July,August,31,1,2018-08-01
294,September 25-26 Meeting - 2018,September,25-26,2018,n,a,25,26,2018-09-26
295,November 7-8 Meeting - 2018,November,7-8,2018,n,a,7,8,2018-11-08


In [475]:
#df.to_csv('fed_dates_raw.csv', index = False)
df1= df[df['month'] == 'None']
df1

,dates,month,day,year
0,Jan/Feb 31-1 Meeting - 2017,None,31-1,2017
6,Oct/Nov 31-1 Meeting - 2017,None,31-1,2017
4,Jul/Aug 31-1 Meeting - 2018,None,31-1,2018


In [394]:

url = f"https://www.federalreserve.gov/monetarypolicy/fomchistorical{2010}.htm"

item = 'h5'

def get_html_item(url, item):

    # Make a GET request to fetch the raw HTML content
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the header
        headers = soup.find_all(item)

    else: 
        print("Unsuccessful Call: ", response.status_code)

    return headers


get_html_item(url, item)

[<h5>January 26-27 Meeting - 2010</h5>,
 <h5>March 16 Meeting - 2010</h5>,
 <h5>April 27-28 Meeting - 2010</h5>,
 <h5>May 9 Conference Call - 2010</h5>,
 <h5>June 22-23 Meeting - 2010</h5>,
 <h5>August 10 Meeting - 2010</h5>,
 <h5>September 21 Meeting - 2010</h5>,
 <h5>October 15 Conference Call - 2010</h5>,
 <h5>November 2-3 Meeting - 2010</h5>,
 <h5>December 14 Meeting - 2010</h5>]

In [402]:
def headers_to_dates(headers):

    dates = {}
    dates_list = []

    for header in headers:

        items = header.text.split(' ')
        
        month = items[0]
        days = items[1]
        
        try:
            d1, d2 = days.split("-")
        except ValueError:
            d2 = days 
            
        if '/' in month:
            
            m1,m2 = month.split("/")
            dates[complete_month(m2)] = d2 
            
            day1 = datetime.date(year, month_str_to_num(complete_month(m1)), int(d1))
            day2 = datetime.date(year, month_str_to_num(complete_month(m2)), int(d2))
            dates_list.append(day1)
            dates_list.append(day2)
            
        else:
            
            dates[month] = d2 
            try:
                day1 = datetime.date(year, month_str_to_num(month), int(d1))
                day2 = datetime.date(year, month_str_to_num(month), int(d2))
                dates_list.append(day1)
                dates_list.append(day2)
                
            except UnboundLocalError:
                day2 = datetime.date(year, month_str_to_num(month), int(d2))
                dates_list.append(day2)
        
    return (dates, dates_list)

headers_to_dates(headers)

({'January': '9',
  'February': '6',
  'March': '26',
  'April': '30',
  'May': '14',
  'June': '24',
  'July': '3',
  'August': '20',
  'September': '13',
  'October': '30',
  'November': '5',
  'December': '17'},
 [datetime.date(1991, 1, 9),
  datetime.date(1991, 2, 1),
  datetime.date(1991, 2, 5),
  datetime.date(1991, 2, 6),
  datetime.date(1991, 3, 5),
  datetime.date(1991, 3, 26),
  datetime.date(1991, 4, 5),
  datetime.date(1991, 4, 12),
  datetime.date(1991, 4, 5),
  datetime.date(1991, 4, 30),
  datetime.date(1991, 5, 5),
  datetime.date(1991, 5, 1),
  datetime.date(1991, 5, 5),
  datetime.date(1991, 5, 14),
  datetime.date(1991, 6, 5),
  datetime.date(1991, 6, 10),
  datetime.date(1991, 6, 5),
  datetime.date(1991, 6, 24),
  datetime.date(1991, 7, 2),
  datetime.date(1991, 7, 3),
  datetime.date(1991, 8, 2),
  datetime.date(1991, 8, 5),
  datetime.date(1991, 8, 2),
  datetime.date(1991, 8, 20),
  datetime.date(1991, 9, 2),
  datetime.date(1991, 9, 13),
  datetime.date(1991, 1

In [395]:
headers = get_html_item(url, item)

dates, dates_list = headers_to_dates(headers)

In [390]:
dates = {}
dates_list = []

for header in headers:

    items = header.text.split(' ')
    
    month = items[0]
    days = items[1]
    
    try:
        d1, d2 = days.split("-")
    except ValueError:
        d2 = days 
        
    if '/' in month:
        
        m1,m2 = month.split("/")
        dates[complete_month(m2)] = d2 
        
        day1 = datetime.date(year, month_str_to_num(complete_month(m1)), int(d1))
        day2 = datetime.date(year, month_str_to_num(complete_month(m2)), int(d2))
        dates_list.append(day1)
        dates_list.append(day2)
        
    else:
        
        dates[month] = d2 
        day1 = datetime.date(year, month_str_to_num(month), int(d1))
        day2 = datetime.date(year, month_str_to_num(month), int(d2))
        dates_list.append(day1)
        dates_list.append(day2)
        
        
        #datetime.date(year, month_str_to_num(month), d2)
        
dates_list

[datetime.date(2018, 1, 30),
 datetime.date(2018, 1, 31),
 datetime.date(2018, 3, 20),
 datetime.date(2018, 3, 21),
 datetime.date(2018, 5, 1),
 datetime.date(2018, 5, 2),
 datetime.date(2018, 6, 12),
 datetime.date(2018, 6, 13),
 datetime.date(2018, 7, 31),
 datetime.date(2018, 8, 1),
 datetime.date(2018, 9, 25),
 datetime.date(2018, 9, 26),
 datetime.date(2018, 11, 7),
 datetime.date(2018, 11, 8),
 datetime.date(2018, 12, 18),
 datetime.date(2018, 12, 19)]

In [ ]:
d

In [340]:

def headers_to_dates(headers):

    dates = {}

    for header in headers:

        items = header.text.split(' ')
        
        month = items[0]
        days = items[1]
        
        try:
            d1, d2 = days.split("-")
        except ValueError:
            d2 = days 
        
        if '/' in month:
            
            m1,m2 = month.split("/")
            dates[complete_month(m2)] = d2 
            
        else:
            
            dates[month] = d2 
        
    
    return dates

In [325]:
df= pd.DataFrame(dates, index = [1],columns = dates.keys()).transpose()
df.columns = [str(year)]
df

,2017
February,1
March,15
May,3
June,14
July,26
September,20
November,1
December,13


In [332]:
df = pd.DataFrame(index = calendar.month_name[1:])


df

""
January
February
March
April
May
June
July
August
September
October
